# Test

In [1]:
import sys

In [2]:
project_folder = "../"
sys.path.insert(0, project_folder)
print(sys.path) # view the path and verify

['../', '/Users/federicosiciliano/Desktop/Py_utils/torch_utils/ntb', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python38.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site-packages']


In [3]:
import exp_utils, data_utils, torch_utils

## Configuration

In [4]:
cfg = exp_utils.cfg.load_configuration("config_nn")

In [5]:
data = data_utils.data.load_data(**cfg["data"])

/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:178.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
cfg["model"]["in_channels"] = data["train_x"].shape[1]
cfg["model"]["out_features"] = data["train_y"].shape[1]

In [8]:
loaders = torch_utils.preparation.prepare_data_loaders(data, cfg["model"]["loader_params"])

In [9]:
main_module = torch_utils.model.get_torchvision_model(**cfg["model"])

In [10]:
callbacks = torch_utils.preparation.prepare_callbacks(cfg["model"]["trainer_params"])
already_defined = ["callbacks"]
trainer = torch_utils.preparation.prepare_trainer(**{k:cfg["model"]["trainer_params"][k] for k in cfg["model"]["trainer_params"] if k not in already_defined}, callbacks=callbacks)

Global seed set to 42
Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
loss = torch_utils.preparation.prepare_loss(cfg["model"]["loss"])

In [12]:
optimizer = torch_utils.preparation.prepare_optimizer(**cfg["model"]["optimizer"])

In [13]:
model = torch_utils.process.create_model(main_module, loss, optimizer)

Global seed set to 42


In [14]:
torch_utils.process.train_model(trainer, model, loaders)


  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 11.2 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.701    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [16]:
torch_utils.process.test_model(trainer, model, loaders)

Testing: 0it [00:00, ?it/s]

/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
